In [15]:
import pandas as pd
import json
# from utils.preprocessing import generate_pair_data
import os
import numpy as np
from sklearn.metrics import mean_squared_error


In [ ]:
pairs = {eval(k): v for k,v in dict(json.loads(open("data/info/pairs.json", "r").read())).items()}

for pair in pairs.keys():
    df_A = pd.read_pickle("data/raw/" + pair[0] + ".zip")  # needs pickle5 compression (python 3.8)
    df_B = pd.read_pickle("data/raw/" + pair[1] + ".zip")  # needs pickle5 compression (python 3.8)
    pair_df = generate_pair_data(df_A, df_B, "1999-01-01", "2021-03-01")
    pair_df.to_pickle(f"data/processed/{pair[0].upper()}-{pair[1].upper()}.zip")
    print(f"saved data/processed/{pair[0].upper()}-{pair[1].upper()}.zip")

### TODO: FIX BELOW

In [ ]:
model = ElasticNet()
model.fit(np.array(pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)")),
          np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"]))
pred_diff = model.predict(np.array(pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)")))
len(pred_diff)

In [ ]:
len(np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"]))

In [ ]:
print(np.std(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.mean(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.min(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
print(np.max(np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])))
mean_squared_error(np.array(pair_df.loc["2015-01-01":, "Return Diff (t+1)"]), pred_diff)

In [ ]:
annualized_return(df_A.loc[:,["Simple Return", "Log Return"]])

In [ ]:
from utils.models import GeometricBrownianMotion

GeometricBrownianMotion(1, 0.03, 0.05).simulate()

### Other Models

In [16]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

def make_model(model, param_value):
    # acceptable model
    models = ['svc', 'forest', 'adaboost', 'gradboost', 'nn']
    assert model in models
    
    if model == 'svc': return SVC(C=param_value)
    elif model == 'forest': return RandomForestRegressor(n_estimators=param_value)
    elif model == 'adaboost': return AdaBoostRegressor(n_estimators=param_value)
    elif model == 'gradboost': return GradientBoostingRegressor(n_estimators=param_value)
    # hidden_layer_sizes is a tuple, where the ith element is the size of the ith hidden layer
    # nn = neural network
    elif model == 'nn': return MLPRegressor(hidden_layer_sizes=param_values) 
    else: raise Exception('unacceptable model type')
    
    
modelparams = {
    'svc': [10**x for x in range(-3,4)],
    'forest': range(50,100,10),
    'adaboost': range(50,100,10),
    'gradboost': range(50,100,10),
    'nn': [(x,y) for x in range(50,100,10) for y in range(50,100,10)]
}


In [17]:
data_path = r'data//processed'
data_filenames = os.listdir(data_path)

for data_file in data_filenames:
    pairs = data_file[:-4].split('-')
    print("Pairs Evaluated:", pairs[0], pairs[1])
    
    pair_df = pd.read_pickle(os.path.join(data_path, data_file))
    X = np.array(pair_df.loc["2000-01-01":"2014-12-31", :].drop(columns="Return Diff (t+1)"))
    y = np.array(pair_df.loc["2000-01-01":"2014-12-31", "Return Diff (t+1)"])
    X_test = np.array(pair_df.loc["2015-01-01":, :].drop(columns="Return Diff (t+1)"))
    
    
    for key in modelparams:
        print("Model:", key)
        for param in modelparams[key]:
            print("Parameter Value:", param)
            model = make_model(key, param)
            model.fit(X,y)

            pred = model.predict(X_test)
            mse = mean_squared_error(X_test, pred)
            print("MSE:", mse)
            print()
        print("======================\n")
    
    

Pairs Evaluated: ADX TY
Model: svc
Parameter Value: 0.001


ValueError: Unknown label type: 'continuous'

In [23]:
y

array([ 0.01919701, -0.0016556 , -0.00062528, ..., -0.00256943,
        0.00018649, -0.00330952])

[1, 2, 3]

In [13]:
name = 'XX-YY.zip'
name[:-4].split('-')

['XX', 'YY']